# DI Pre Processing
This notebook converts a video into frames. 
<br>
<br>
![UofC logo](../assets/images/uofc_logo-black.jpg)

In [14]:
#import libraries
import os
from dotenv import load_dotenv
import torch
from torch.utils.data import DataLoader

In [15]:
# reload for module caching
from importlib import reload
import src.dataset.breathing_dataset
reload(src.dataset.breathing_dataset)
import src.transforms.breathing_transforms
reload(src.transforms.breathing_transforms)
import src.deep_learning_models.three_d_cnn
reload(src.deep_learning_models.three_d_cnn)
import src.utilities.train_utilities
reload(src.utilities.train_utilities)

from src.dataset.breathing_dataset import BreathingDataset
from src.dataset.breathing_dataset_config import BreathingDatasetConfig
from src.transforms.breathing_transforms import train_transform
from src.deep_learning_models.three_d_cnn import Basic3DCNN
from src.utilities.train_utilities import train_validate

In [16]:
curr_dir = os.getcwd()

In [17]:
# load root directory
dotenv_path = os.path.join(curr_dir, ".env")
load_dotenv(dotenv_path)
root_path = os.getenv("ROOT_FOLDER")

# load user drive
user_drive = os.getenv("EXPORT_DRIVE")
local_export_path = os.getenv("EXPORT_PATH_LOCAL")
remote_export_path = os.getenv("EXPORT_PATH_REMOTE")
best_model_path = os.getenv("BEST_MODEL_PATH")

## Run functions

In [18]:
# load data
train_path = local_export_path + "/Train"
validation_path = local_export_path + "/Test"

train_dataset_config = BreathingDatasetConfig(train_path, "Without Blankets", None, train_transform, 100)
val_dataset_config = BreathingDatasetConfig(validation_path, "Without Blankets", None, train_transform, 100)

In [19]:
# Dataset
train_dataset_without_blankets = BreathingDataset(train_dataset_config)
val_train_dataset_without_blankets = BreathingDataset(val_dataset_config)

In [20]:
# DataLoader
train_loader = DataLoader(train_dataset_without_blankets, batch_size=2, shuffle=True) 
val_loader = DataLoader(val_train_dataset_without_blankets, batch_size=2, shuffle=True) 

In [21]:
# Parameters
epochs = 1
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_batch_size = 64
val_batch_size = 64
patience = 5
verbose = True

In [22]:
# Model Instance
model = Basic3DCNN()

### Testing sliding window (will delete later)

In [23]:
def train_model(model, dataset, window_size, step_size, epochs, optimizer, loss_function):
    model.train()  # Set the model to training mode
    
    for epoch in range(epochs):
        total_loss = 0
        for images, label in dataset.sliding_window_generator(window_size, step_size):
            # Ensure 'images' has the correct shape [batch_size, channels, depth, height, width]
            images = images.unsqueeze(0) if images.dim() == 4 else images  # Adjust for missing batch dimension
            
            # Convert 'label' to a tensor and ensure it has the correct shape
            label = torch.tensor([label], dtype=torch.float32) if isinstance(label, int) else label

            optimizer.zero_grad()  # Zero the gradients
            
            # Forward pass
            output = model(images)
            loss = loss_function(output, label.unsqueeze(1))  # Adjust 'label' shape for loss calculation
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()

        print(f'Epoch {epoch+1}, Loss: {total_loss}')


In [24]:
import torch.nn as nn
import torch.optim as optim

In [25]:
# Assuming model, optimizer, and loss_function are already defined
window_size = 5  # Number of frames in each window
step_size = 2    # Steps between the start of each window
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

In [26]:
train_model(model, train_dataset_without_blankets, window_size, step_size, epochs=10, optimizer=optimizer, loss_function=loss_function)

RuntimeError: Given input size: (128x1x112x112). Calculated output size: (128x0x56x56). Output size is too small

In [ ]:
# """ Train and Validate using data loader"""
# train_validate(
#     model=model,
#     train_loader=train_loader,
#     val_loader=val_loader,
#     epochs=epochs,
#     learning_rate=learning_rate,
#     best_model_path=best_model_path,
#     device=device,
#     patience=patience,
#     verbose=verbose
# )